 Erstellen Sie eine Python-Funktion, welche die Daten aus der Datei einliest (verwenden Sie
 dazu die Numpy-Funktion genfromtxt()), von den Daten die ersten 1000 Werte überspringt
 und aus den nächsten Werten (nehmen Sie eine sinnvolle Anzahl von Messungen) den
 Mittelwert und die Standardabweichung berechnet (der Einschwingvorgang wird dadurch
 ignoriert). Vergleichen Sie die Ergebnisse mit den Werten aus Ihrer gemessenen Tabelle.
 Stellen Sie die gefundene Übertragungsfunktion bzw. Kennlinie im Notebook graphisch mit
 Python bzw. Matplotlib dar.

In [1]:
import numpy as np

# use genfromtxt to read the data from 11 files called V1_10.csv to V1_70.csv
def read_data():
    data = []
    for i in range(10, 71):
        data.append(np.genfromtxt(f"V1_{i}.csv", delimiter=",", skip_header=1000))
    return data
